<a href="https://colab.research.google.com/github/kabirwalia8300/HMT_vs_NMT/blob/main/HMT_NMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Classify if a sentence was translated by a machine or a human

In [1]:
# mounting
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# General imports
import collections
import json
import string
import re
import numpy as np
import pandas as pd
# import torch
# import torch.nn as nn
# import urllib


Parse in data


In [4]:
from collections import defaultdict

# training 
f=open("train.txt",'r')
train_txt = []
temp = []
for line in f:
    if line=='\n':
        train_txt.append(temp)
        temp = []
        continue
    temp.append(line)

# training dataframe
train_df = pd.DataFrame(train_txt, columns=['orig', 'ref', 'candidate', 'score', 'label'])


# testing 
t=open("test.txt",'r')
test_txt = []
temp = []
for line in t:
    if line=='\n':
        test_txt.append(temp)
        temp = []
        continue
    temp.append(line)

# testing dataframe
test_df = pd.DataFrame(test_txt, columns=['orig', 'ref', 'candidate', 'score', 'label'])

In [5]:
# training
train_df['orig']=train_df.orig.apply(lambda x: x.strip('\n'))
train_df['candidate']=train_df.candidate.apply(lambda x: x.strip('\n'))
train_df['label']=train_df.label.apply(lambda x: x.strip('\n'))
train_df['score']=train_df.score.apply(lambda x: x.strip('\n'))

# testing
test_df['orig']=test_df.orig.apply(lambda x: x.strip('\n'))
test_df['candidate']=test_df.candidate.apply(lambda x: x.strip('\n'))
test_df['label']=test_df.label.apply(lambda x: x.strip('\n'))
test_df['score']=test_df.score.apply(lambda x: x.strip('\n'))

In [6]:
train_df.head()

,orig,ref,candidate,score,label
0,巴林 公主 下 嫁 美 大兵 惊 世 婚姻 五 年 宣告 破裂,"bahraini princess marries us soldier , astonis...",bahraini princess marries a u.s. soldier ; ast...,0.3125,H
1,"巴林 公主 梅 丽 安 ・ 哈 里 发 下 嫁 美国 陆 战 队 大兵 强 生 , 曾 获 ...",the star-crossed marriage between bahraini pri...,u.s. television stations had once feted the ma...,0.6531,H
2,"梅 丽 安 是 海湾 国家 巴林 王室 的 成员 , 强 生 冒 着 赔 上 军旅 生涯 的...",meriam is a member of the gulf country bahrain...,"meri gulf state of bahrain , the royal family ...",0.3784,M
3,"不过 , 据 拉 斯 维 加 斯 评论 报 报导 , 才 过 了 五 年 , 两 人 就 对...",but the las vegas review-journal reported that...,"however , according to the las vegas , comment...",0.3646,M
4,"他们 两 人 在 一九九九年 相遇 , 当时 强 生 还是 职业 军人 , 派 驻 在 巴林 .",the pair met in 1999 when career military man ...,"the two met in 1999 , when johnson was still a...",0.7778,H


In [7]:
test_df.head()

,orig,ref,candidate,score,label
0,"发言人 说 , 世人 共 知 , 中国 的 发展 不仅 符合 13 亿 中国人民 的 根本利...",the spokesman noted that it is known to all th...,"the spokesman said , the world , a total of 13...",0.5100,M
1,"在 国际 政治 舞台上 , 中国 是 一个 负责任 的 国家 , 对 维护 世界和平 与 稳...","in the international political arena , china i...","in the international political arena , china i...",0.5996,H
2,"在 核 不 扩散 问题 上 , 中国 始终 严格 履行 有关 国际公约 , 坚决 反对 任何...","on the issue of nuclear non-proliferation , ch...",on the issue of non-proliferation of nuclear w...,0.6667,H
3,"发言人 指出 , 令人 遗憾 的 是 , 海德 先生 无视 客观 事实 , 在 中美 关系 ...",the spokesman points out that it is regrettabl...,the spokesman stated that it was regretful tha...,0.5326,H
4,中国 最 大规模 的 海水 淡化 工程 落户 舟山,china's largest seawater desalination project ...,china's largest seawater desalination project ...,0.6014,H


In [8]:
train_df['bi_text'] = train_df["orig"].astype(str) +' '+ train_df["candidate"].astype(str)
test_df['bi_text'] = test_df["orig"].astype(str) +' '+ test_df["candidate"].astype(str)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

Get largest sequence length after splitting combined sentences on spaces. (Probably, not the best method because chinese text isn't space separated)

In [11]:
print(f"Training: {max(train_df['bi_text'].apply(lambda x: len(x.split())))}")
print(f"Testing: {max(test_df['bi_text'].apply(lambda x: len(x.split())))}")

Training: 170
Testing: 194


In [12]:
# train test split using numpy - use 20% of training data as validation
msk = np.random.rand(len(train_df)) < 0.8
train = train_df[msk]
train = train.reset_index(drop=True)
val = train_df[~msk]
val = val.reset_index(drop=True)

In [13]:
# install simpletransformers
# !pip install tokenizers==0.9.4
# !pip install simpletransformers

# check installed version
!pip freeze | grep simpletransformers
!pip freeze | grep tokenizers
# simpletransformers==0.28.2

simpletransformers==0.60.6
tokenizers==0.9.4


In [14]:
train = train[['orig', 'candidate', 'label']]
val = val[['orig', 'candidate', 'label']]
train['label_t'] = train.label.apply(lambda x: 0 if x=='H' else 1)
val['label_t'] = val.label.apply(lambda x: 0 if x=='H' else 1)
del train['label']
del val['label']

In [15]:
test = test_df[['orig', 'candidate', 'label']]
test['label_t'] = test.label.apply(lambda x: 0 if x=='H' else 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
test.head()

,orig,candidate,label,label_t
0,"发言人 说 , 世人 共 知 , 中国 的 发展 不仅 符合 13 亿 中国人民 的 根本利...","the spokesman said , the world , a total of 13...",M,1
1,"在 国际 政治 舞台上 , 中国 是 一个 负责任 的 国家 , 对 维护 世界和平 与 稳...","in the international political arena , china i...",H,0
2,"在 核 不 扩散 问题 上 , 中国 始终 严格 履行 有关 国际公约 , 坚决 反对 任何...",on the issue of non-proliferation of nuclear w...,H,0
3,"发言人 指出 , 令人 遗憾 的 是 , 海德 先生 无视 客观 事实 , 在 中美 关系 ...",the spokesman stated that it was regretful tha...,H,0
4,中国 最 大规模 的 海水 淡化 工程 落户 舟山,china's largest seawater desalination project ...,H,0


In [17]:
train=train.rename(columns={'orig':'text_a', 'candidate': 'text_b', 'label_t':'labels'})
val =val.rename(columns={'orig':'text_a', 'candidate': 'text_b', 'label_t':'labels'})

In [18]:
train.head()

,text_a,text_b,labels
0,巴林 公主 下 嫁 美 大兵 惊 世 婚姻 五 年 宣告 破裂,bahraini princess marries a u.s. soldier ; ast...,0
1,"巴林 公主 梅 丽 安 ・ 哈 里 发 下 嫁 美国 陆 战 队 大兵 强 生 , 曾 获 ...",u.s. television stations had once feted the ma...,0
2,"梅 丽 安 是 海湾 国家 巴林 王室 的 成员 , 强 生 冒 着 赔 上 军旅 生涯 的...","meri gulf state of bahrain , the royal family ...",1
3,"不过 , 据 拉 斯 维 加 斯 评论 报 报导 , 才 过 了 五 年 , 两 人 就 对...","however , according to the las vegas , comment...",1
4,"他们 两 人 在 一九九九年 相遇 , 当时 强 生 还是 职业 军人 , 派 驻 在 巴林 .","the two met in 1999 , when johnson was still a...",0


In [19]:
test =test.rename(columns={'orig':'text_a', 'candidate': 'text_b', 'label_t':'labels'})
del test['label']

In [20]:
test.head()

,text_a,text_b,labels
0,"发言人 说 , 世人 共 知 , 中国 的 发展 不仅 符合 13 亿 中国人民 的 根本利...","the spokesman said , the world , a total of 13...",1
1,"在 国际 政治 舞台上 , 中国 是 一个 负责任 的 国家 , 对 维护 世界和平 与 稳...","in the international political arena , china i...",0
2,"在 核 不 扩散 问题 上 , 中国 始终 严格 履行 有关 国际公约 , 坚决 反对 任何...",on the issue of non-proliferation of nuclear w...,0
3,"发言人 指出 , 令人 遗憾 的 是 , 海德 先生 无视 客观 事实 , 在 中美 关系 ...",the spokesman stated that it was regretful tha...,0
4,中国 最 大规模 的 海水 淡化 工程 落户 舟山,china's largest seawater desalination project ...,0


In [21]:
sum(train_df['label']=='H')

311

In [22]:
sum(train_df['label']=='M')

272

## Monolingual Model: Bert

In [ ]:
from simpletransformers.classification import (
    ClassificationModel, ClassificationArgs
)

# define hyperparameter
model_args = ClassificationArgs(num_train_epochs=5, overwrite_output_dir=True)

# # Create a ClassificationModel
mono_bert = ClassificationModel(
    "bert", "bert-base-cased",
    num_labels=2,
    args=model_args,
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
mono_train = train[['text_b', 'labels']]
mono_train = mono_train.rename(columns={'text_b': 'text'})
mono_val = val[['text_b', 'labels']]
mono_val = mono_val.rename(columns={'text_b': 'text'})

In [ ]:
mono_train.head()

,text,labels
0,u.s. television stations had once feted the ma...,0
1,"meri gulf state of bahrain , the royal family ...",1
2,"however , according to the las vegas , comment...",1
3,"the two met in 1999 , when johnson was still a...",0
4,"after he met this teenage princess , the pair ...",0


Training Monolingual model


In [ ]:
mono_bert.train_model(mono_train)

  0%|          | 0/468 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/59 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running Epoch 1 of 5:   0%|          | 0/59 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/59 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/59 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/59 [00:00<?, ?it/s]

(295, 0.14649033026125527)

Evaluate Monolingual model

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='macro')

result, model_outputs, wrong_predictions = mono_bert.eval_model(mono_val, f1=f1_multiclass, acc=accuracy_score)

  0%|          | 0/115 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
result

{'acc': 0.9130434782608695,
 'auprc': 0.9676997986755728,
 'auroc': 0.966858353510896,
 'eval_loss': 0.5110749937754008,
 'f1': 0.9129842615012107,
 'fn': 5,
 'fp': 5,
 'mcc': 0.8259685230024213,
 'tn': 54,
 'tp': 51}

## Bilingual Model: Sentence Pair Classification using mBert

In [25]:
from simpletransformers.classification import (
    ClassificationModel, ClassificationArgs
)

# define hyperparameter
model_args = ClassificationArgs(num_train_epochs=7, max_seq_length=256, overwrite_output_dir=True,)

# # Create a ClassificationModel
m_bert = ClassificationModel(
    "bert", "bert-base-multilingual-cased",
    num_labels=2,
    args=model_args,
)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Training Bilingual Model

In [26]:
m_bert.train_model(train)

  0%|          | 0/464 [00:00<?, ?it/s]

Epoch:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 0 of 7:   0%|          | 0/58 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running Epoch 1 of 7:   0%|          | 0/58 [00:00<?, ?it/s]

...

Running Epoch 3 of 7:   0%|          | 0/58 [00:00<?, ?it/s]

Running Epoch 4 of 7:   0%|          | 0/58 [00:00<?, ?it/s]

Running Epoch 5 of 7:   0%|          | 0/58 [00:00<?, ?it/s]

Running Epoch 6 of 7:   0%|          | 0/58 [00:00<?, ?it/s]

(406, 0.20264032484763966)

Evaluate Bilingual Model

In [27]:
from sklearn.metrics import f1_score, accuracy_score

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='macro')

result, model_outputs, wrong_predictions = m_bert.eval_model(val, f1=f1_multiclass, acc=accuracy_score)

  0%|          | 0/119 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/15 [00:00<?, ?it/s]

In [28]:
result

{'acc': 0.8991596638655462,
 'auprc': 0.978080678677766,
 'auroc': 0.9796610169491526,
 'eval_loss': 0.5839654981546725,
 'f1': 0.8988095238095237,
 'fn': 9,
 'fp': 3,
 'mcc': 0.8022308235600581,
 'tn': 57,
 'tp': 50}

## Test Set

Monolingual Model

In [ ]:
mono_test = test[['text_b', 'labels']]
mono_test = mono_test.rename(columns={'text_b': 'text'})

In [ ]:
result, model_outputs, wrong_predictions = mono_bert.eval_model(mono_test, f1=f1_multiclass, acc=accuracy_score)
result

  0%|          | 0/173 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/22 [00:00<?, ?it/s]

{'acc': 0.9075144508670521,
 'auprc': 0.9668154946810085,
 'auroc': 0.9634816403109087,
 'eval_loss': 0.5627917768453533,
 'f1': 0.906989247311828,
 'fn': 10,
 'fp': 6,
 'mcc': 0.8149592581651243,
 'tn': 85,
 'tp': 72}

Bilingual Model

In [29]:
result, model_outputs, wrong_predictions = m_bert.eval_model(test, f1=f1_multiclass, acc=accuracy_score)
result

  0%|          | 0/173 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/22 [00:00<?, ?it/s]

{'acc': 0.884393063583815,
 'auprc': 0.9624758412129071,
 'auroc': 0.9683060841597427,
 'eval_loss': 0.7233396952545842,
 'f1': 0.8826641345632121,
 'fn': 16,
 'fp': 4,
 'mcc': 0.774105464990605,
 'tn': 87,
 'tp': 66}